<a href="https://colab.research.google.com/github/leonardobocci/ml-stock-market/blob/main/0.master_thesis_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pipeline Steps:
1. Load data for all ETFs
2. Perform column calculations (eg. returns and indicators)
3. Handle Outliers
3. Transform data (eg. dates to cyclical coordinates)
4. Enforce Stationarity
5. Save Stationary Data

# Libraries and Data Loading

In [ ]:
%%capture
!pip install feature_engine
!pip install statsmodels --upgrade

In [ ]:
%%capture
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
!tar xvzf ta-lib-0.4.0-src.tar.gz
import os
os.chdir('ta-lib') # Can't use !cd in colab
!./configure --prefix=/usr
!make
!make install
os.chdir('../')
!pip install TA-Lib
!pip install -U git+https://github.com/twopirllc/pandas-ta
import talib, pandas_ta as ta

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit 
from feature_engine.creation import CyclicalFeatures
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

workbook = gc.open('all_etfs_OHLCV')
sheet_titles = []
for sheet in workbook.worksheets():
  sheet_titles.append(sheet.title)

dict_of_sheets = {}
for sheet_title in sheet_titles:
  sheet = workbook.worksheet(sheet_title)
  values = sheet.get_all_values()
  dict_of_sheets[sheet_title] = values

keys = list(dict_of_sheets)
etfs = {}
for etf in keys:
  etfs[etf] = pd.DataFrame.from_records(dict_of_sheets[etf])
  cols = ['date', 'open', 'high', 'low', 'close', 'volume']
  etfs[etf].columns = cols
  etfs[etf] = etfs[etf][1:]
  etfs[etf]['date'] = pd.to_datetime(etfs[etf]['date'], format="%d/%m/%Y")
  etfs[etf].set_index('date', inplace=True)
  etfs[etf] = etfs[etf].apply(pd.to_numeric, errors='coerce')
  etfs[etf] = etfs[etf].fillna(method="ffill")
  etfs[etf].dropna(axis = 0, inplace = True)    

# Indicators

In [ ]:
def add_ta_indicators(original_df):
  df = original_df.copy()
  #SMA
  df['sma_50'] = df.ta.sma(close=df.close, length=50)
  df['sma_200'] = df.ta.sma(close=df.close, length=200)
  df.loc[(df.sma_50 > df.sma_200) & (df.sma_50.shift(1) <= df.sma_200.shift(1)), 'sma_cross'] = 1
  df.loc[(df.sma_50 < df.sma_200) & (df.sma_50.shift(1) >= df.sma_200.shift(1)), 'sma_cross'] = -1
  df.sma_cross.fillna(0, inplace=True)

  #WMA
  df['wma_50'] = df.ta.wma(close=df.close, length=50)
  df['wma_200'] = df.ta.wma(close=df.close, length=200)
  df.loc[(df.wma_50 > df.wma_200) & (df.wma_50.shift(1) <= df.wma_200.shift(1)), 'wma_cross'] = 1
  df.loc[(df.wma_50 < df.wma_200) & (df.wma_50.shift(1) >= df.wma_200.shift(1)), 'wma_cross'] = -1
  df.wma_cross.fillna(0, inplace=True)

  #EMA
  df['ema_12'] = df.ta.ema(close=df.close, length=12)
  df['ema_26'] = df.ta.ema(close=df.close, length=26)
  df.loc[(df.ema_12 > df.ema_26) & (df.ema_12.shift(1) <= df.ema_26.shift(1)), 'ema_cross'] = 1
  df.loc[(df.ema_12 < df.ema_26) & (df.ema_12.shift(1) >= df.ema_26.shift(1)), 'ema_cross'] = -1
  df.ema_cross.fillna(0, inplace=True)

  #DEMA
  df['dema_50'] = df.ta.dema(close=df.close, length=50)
  df['dema_200'] = df.ta.dema(close=df.close, length=200)
  df.loc[(df.dema_50 > df.dema_200) & (df.dema_50.shift(1) <= df.dema_200.shift(1)), 'dema_cross'] = 1
  df.loc[(df.dema_50 < df.dema_200) & (df.dema_50.shift(1) >= df.dema_200.shift(1)), 'dema_cross'] = -1
  df.dema_cross.fillna(0, inplace=True)

  #TRIMA
  df['trima_50'] = df.ta.trima(close=df.close, length=50)
  df['trima_200'] = df.ta.trima(close=df.close, length=200)
  df.loc[(df.trima_50 > df.trima_200) & (df.trima_50.shift(1) <= df.trima_200.shift(1)), 'trima_cross'] = 1
  df.loc[(df.trima_50 < df.trima_200) & (df.trima_50.shift(1) >= df.trima_200.shift(1)), 'trima_cross'] = -1
  df.trima_cross.fillna(0, inplace=True)

  #HULL
  df['hull_50'] = df.ta.hma(close=df.close, length=50)
  df['hull_200'] = df.ta.hma(close=df.close, length=200)
  df['hull_12'] = df.ta.hma(close=df.close, length=12)
  df['hull_26'] = df.ta.hma(close=df.close, length=26)
  df.loc[(df.hull_50 > df.hull_200) & (df.hull_50.shift(1) <= df.hull_200.shift(1)), 'hull_slow_cross'] = 1
  df.loc[(df.hull_50 < df.hull_200) & (df.hull_50.shift(1) >= df.hull_200.shift(1)), 'hull_slow_cross'] = -1
  df.hull_slow_cross.fillna(0, inplace=True)
  df.loc[(df.hull_12 > df.hull_26) & (df.hull_12.shift(1) <= df.hull_26.shift(1)), 'hull_fast_cross'] = 1
  df.loc[(df.hull_12 < df.hull_26) & (df.hull_12.shift(1) >= df.hull_26.shift(1)), 'hull_fast_cross'] = -1
  df.hull_fast_cross.fillna(0, inplace=True)

  #STOCH
  stoch = df.ta.stoch(close=df.close,high=df.high, low=df.low)
  stoch.columns = ['k', 'd']
  df['stoch_d'] = stoch.d
  df['stoch_k'] = stoch.k

  #RSI
  df['rsi'] = df.ta.rsi(close=df.close)

  #MACD
  macd = df.ta.macd(close=df.close)
  macd.columns = ['macd', 'histogram', 'signal']
  df['macd_hist'] = macd.histogram
  df.loc[(df.macd_hist > 0) & (df.macd_hist.shift(1) <= 0), 'macd_hist_cross'] = 1
  df.loc[(df.macd_hist < 0) & (df.macd_hist.shift(1) >= 0), 'macd_hist_cross'] = -1
  df.macd_hist_cross.fillna(0, inplace=True)

  #WILLIAMS
  df['williams_r'] = df.ta.willr(close=df.close,high=df.high, low=df.low)

  #ADO
  df['ado'] = df.ta.adosc(close=df.close,high=df.high, low=df.low, open=df.open, volume=df.volume)

  #CCI
  df['cci'] = df.ta.ad(close=df.close,high=df.high, low=df.low)

  #ROC
  df['roc'] = df.ta.roc(close=df.close, length=12)

  #DI
  #Here the disparity index may use any type of moving average, hull could be a better option than sma.
  df['di'] = df.ta.sma(close=df.close, length=14)
  df['di'] = (df.close - df.di) / df.di * 100

  #PPO
  ppo = df.ta.ppo(close=df.close)
  ppo.columns = ['ppo', 'histogram', 'signal']
  df['ppo_hist'] = ppo.histogram

  #PVO
  pvo = df.ta.pvo(volume=df.volume)
  pvo.columns = ['pvo', 'histogram', 'signal']
  df['pvo_hist'] = pvo.histogram

  #PSY
  df['psy'] = df.ta.psl(close=df.close, open=df.open)

  #ADX
  adx = df.ta.adx(high=df.high, low=df.low)
  adx.columns = ['adx', 'dmp', 'dmm']
  adx['dmi'] = abs(adx.dmp - adx.dmm) / abs(adx.dmp + adx.dmm) * 100
  df['adx'] = adx.loc[:, 'adx']
  df['diplus'] = adx.loc[:, 'dmp']
  df['diminus'] = adx.loc[:, 'dmm']
  df['dmi'] = adx.loc[:, 'dmi']

  #OBV
  df['obv'] = df.ta.obv(close=df.close, volume=df.volume)

  #KLINGER
  klinger = df.ta.kvo(close=df.close, volume=df.volume, high=df.high, low=df.low)
  klinger.columns = ['klo', 'signal']
  klinger['histogram'] = klinger.klo - klinger.signal
  df['klo_hist'] = klinger.histogram

  #MFI
  df['mfi'] = df.ta.mfi(close=df.close, volume=df.volume, high=df.high, low=df.low)

  #CMF
  df['cmf'] = df.ta.cmf(close=df.close, volume=df.volume, high=df.high, low=df.low, open=df.open)

  #ATR
  df['atr'] = df.ta.atr(close=df.close, high=df.high, low=df.low)

  #NATR
  df['natr'] = df.ta.natr(close=df.close, high=df.high, low=df.low)

  #BOLLINGER
  bollinger = df.ta.bbands(close=df.close)
  bollinger.columns = ['lower', 'mid', 'upper', 'bandwidth', 'percent']
  df['bolu'] = bollinger.upper
  df['bold'] = bollinger.lower
  df['bol_width'] = bollinger.bandwidth
  df['bol_pct'] = bollinger.percent

  #KELTNER
  keltner = df.ta.kc(close=df.close, high=df.high, low=df.low)
  keltner.columns = ['lower', 'basis', 'upper']
  df['kelu'] = keltner.upper
  df['keld'] = keltner.lower

  #ACCBANDS
  accbands = df.ta.accbands(close=df.close, high=df.high, low=df.low)
  accbands.columns = ['lower', 'middle', 'upper']
  df['abau'] = accbands.upper
  df['abal'] = accbands.lower

  #ENVELOPE
  envelope = df.ta.sma(close=df.close, length=20)
  df['maeu'] = envelope * 1.05
  df['maed'] = envelope * 0.95

  #NVI PVI
  df['nvi'] = df.ta.nvi(close=df.close, volume=df.volume)
  df['pvi'] = df.ta.pvi(close=df.close, volume=df.volume)

  #VWMA
  df['vwma'] = df.ta.vwma(close=df.close, volume=df.volume)

  #PSAR
  psar = df.ta.psar(close=df.close, high=df.high, low=df.low)
  psar.columns = ['long', 'short', 'acceleration', 'reversal']
  df['long_psar'] = psar.long
  df['short_psar'] = psar.short
  df['psar_acc'] = psar.acceleration
  df.long_psar.fillna(df.close, inplace=True)
  df.short_psar.fillna(df.close, inplace=True)
  df['psar_hist'] = (df.close - df.short_psar) + (df.close - df.long_psar)
  df['psar_cross'] = 0
  df.loc[(df.psar_hist > 0) & (df.psar_hist.shift(1) <= 0), 'psar_cross'] = 1
  df.loc[(df.psar_hist < 0) & (df.psar_hist.shift(1) >= 0), 'psar_cross'] = -1
  df.drop(['long_psar', 'short_psar'], axis=1, inplace=True)

  #APO
  df['apo'] = df.ta.apo(close=df.close)

  #AROON
  aroon = df.ta.aroon(close=df.close)
  aroon.columns = ['positive', 'negative', 'oscillator']
  df['arp'] = aroon.positive
  df['arn'] = aroon.negative
  df['aro'] = aroon.oscillator

  #CMO
  df['cmo'] = df.ta.cmo(close=df.close)

  #CFO
  df['cfo'] = df.ta.cfo(close=df.close)

  #COCU
  df['cocu'] = df.ta.coppock(close=df.close)

  #DPO
  df['dpo'] = df.ta.dpo(close=df.close, lookahead=False)

  #EOM
  df['eom'] = df.ta.eom(close=df.close, high=df.high, low=df.low, volume=df.volume)

  #FI
  df['fi'] = df.ta.efi(close=df.close, volume=df.volume)

  #MI
  df['mi'] = df.ta.massi(high=df.high, low=df.low)

  #TP
  df['tp'] = df.ta.hlc3(high=df.high, low=df.low, close=df.close)

  return df

In [ ]:
ta_etfs = {}
for etf in keys:
  ta_etfs[etf] = add_ta_indicators(original_df=etfs[etf])

For dummy columns, 1 = buy signal, -1 = sell signal and 0 = no signal

# Transformations

Log returns is the dependent variable, day/month/year are extracted from the dates variable.

Date cyclical transformation:
To retain the infromation that days and months are cyclical, they are transformed using sin-cos coordinates.

All columns need to be shifted by 1 to avoid data leakage. We need to use past information, not same day to predict returns.

In [ ]:
cyclical = CyclicalFeatures(variables=['day', 'month'])

for etf in keys:
  #Returns
  ta_etfs[etf]['log_returns'] = np.log(ta_etfs[etf].close) - np.log(ta_etfs[etf].close.shift(1))

  #Date Features
  ta_etfs[etf]['day'] = ta_etfs[etf].index.day 
  ta_etfs[etf]['month'] = ta_etfs[etf].index.month 
  ta_etfs[etf]['year'] = ta_etfs[etf].index.year 

  #Date cyclical transformation
  ta_etfs[etf] = cyclical.fit_transform(ta_etfs[etf])

  #Shifting
  mask = ~(ta_etfs[etf].columns.isin(['log_returns']))
  cols_to_shift = ta_etfs[etf].columns[mask]
  ta_etfs[etf][cols_to_shift] = ta_etfs[etf][cols_to_shift].shift(1)
  ta_etfs[etf]['last_log_return'] = ta_etfs[etf].log_returns.shift(1)

  #NA
  ta_etfs[etf].dropna(inplace=True)

# Outlier Handling

Chosen method: Flagging (dummy variable) through IQR method.

Preferred to other methods such as imputation or deletion to preserve information. Assumption: Outliers in the data occur because of external factors that influence the DGP, and not because of data quality errors.

In [ ]:
for etf in keys:
  q3 = ta_etfs[etf].log_returns.quantile(0.75)
  q1 = ta_etfs[etf].log_returns.quantile(0.25)
  iqr = q3- q1

  cutoff = 3
  lower = q1 - cutoff * iqr
  upper = q3 + cutoff * iqr

  #-1 for low values, 0 for normal, 1 for high values
  ta_etfs[etf].loc[ta_etfs[etf]['log_returns'] > upper, 'outlier'] = 1
  ta_etfs[etf].loc[ta_etfs[etf]['log_returns'] < lower, 'outlier'] = -1
  ta_etfs[etf].outlier.fillna(0, inplace=True)

# Drop Nulls

In [ ]:
for etf in keys:
  ta_etfs[etf] = ta_etfs[etf].dropna()

#Descriptive Statistics

In [ ]:
base_description = {}
for etf in keys:
  base_description[etf] = ta_etfs[etf][[
      'log_returns', 'outlier', 'open', 'high', 'low', 'close', 'volume'
  ]].describe()

full_dataset_description = {}
for etf in keys:
  full_dataset_description[etf] = ta_etfs[etf].describe()

#Testing Stationarity

In [ ]:
excluded_cols = ['sma_cross','wma_cross','ema_cross','dema_cross',
                 'trima_cross','hull_slow_cross', 'hull_fast_cross',
                 'macd_hist_cross','psar_cross', 'day', 'month','year',
                 'day_sin', 'day_cos','month_sin', 'month_cos', 'outlier']
columns = [x for x in ta_etfs[etf].columns if x not in excluded_cols]

In [ ]:
%%capture
adf_list = []
kpss_list = []
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
def kpss_test(timeseries):
    kpsstest = kpss(timeseries, regression="c", nlags='auto')
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    if kpss_output['p-value'] < 0.05:
      kpss_list.append(False)
    else:
      kpss_list.append(True)

def adf_test(timeseries):
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    if dfoutput['p-value'] < 0.05:
      adf_list.append(True)
    else:
      adf_list.append(False)

stationarity={}
for etf in keys:
  for col in columns: 
    adf_test(ta_etfs[etf][col].dropna())
    kpss_test(ta_etfs[etf][col].dropna())
  stationarity[etf]= pd.DataFrame(
    {'Feature':columns,
     'Stationary_ADF':adf_list,
     'Stationary_KPSS':kpss_list})
  kpss_list = []
  adf_list = []

Case 1: Both tests conclude that the series is not stationary - The series is not stationary;

Case 2: Both tests conclude that the series is stationary - The series is stationary;

Case 3: KPSS indicates stationarity and ADF indicates non-stationarity - The series is trend stationary. Trend needs to be removed to make series strict stationary. The detrended series is checked for stationarity;

Case 4: KPSS indicates non-stationarity and ADF indicates stationarity - The series is difference stationary. Differencing is to be used to make series stationary. The differenced series is checked for stationarity.

In [ ]:
%%capture
stationary_etfs = {}
for etf in keys:
  stationary_cols =  stationarity[etf].loc[(stationarity[etf].Stationary_ADF == True) & (stationarity[etf].Stationary_KPSS == True), 'Feature'].to_list()
  non_stationary_cols =  stationarity[etf].loc[(stationarity[etf].Stationary_ADF == False) | (stationarity[etf].Stationary_KPSS == False), 'Feature'].to_list()

  stationary = ta_etfs[etf].copy()
  adf_list = []
  kpss_list = []
  persistent_non_stationary = []
  excluded_cols = ['sma_cross','wma_cross','ema_cross','dema_cross',
                  'trima_cross','hull_slow_cross', 'hull_fast_cross',
                  'macd_hist_cross','psar_cross', 'day', 'month',
                  'year', 'day_sin', 'day_cos','month_sin', 'month_cos']
  columns = [x for x in stationary.columns if x not in excluded_cols]
  for col in columns:
    tested_series = []
    exit=False
    if col in non_stationary_cols:
      new_col = 'd_' + col
      tested_series = stationary[col].diff(periods=1) 
      adf_test(tested_series.dropna())
      kpss_test(tested_series.dropna())
      if (adf_list[-1]==True) & (kpss_list[-1]==True):
        exit=True
      if exit==False and ~(stationary[col].values<=0).any():
        new_col = 'log_' + col
        tested_series = np.log(stationary[col])
        adf_test(tested_series.dropna())
        kpss_test(tested_series.dropna())
        if (adf_list[-1]==True) & (kpss_list[-1]==True):
          exit=True
      if exit==False:
        new_col = 'exp_' + col
        tested_series = np.exp(stationary[col]/10000000000)
        adf_test(tested_series.dropna())
        kpss_test(tested_series.dropna())
        if (adf_list[-1]==True) & (kpss_list[-1]==True):
          exit=True
      if exit==False:
        for i in range(2,4):
          new_col = f'd_{i}_' + col
          tested_series = stationary[col].diff(periods=i) 
          adf_test(tested_series.dropna())
          kpss_test(tested_series.dropna())
          if (adf_list[-1]==True) & (kpss_list[-1]==True):
            exit=True
            break
      if exit==False and ~(stationary[col].values<=0).any():
        new_col = 'log_d_' + col
        tested_series = np.log(stationary[col]) - np.log(stationary[col].shift(1))
        adf_test(tested_series.dropna())
        kpss_test(tested_series.dropna())
        if (adf_list[-1]==True) & (kpss_list[-1]==True):
          exit=True
      if exit==False:
        new_col = 'exp_d_' + col
        tested_series = np.exp(stationary[col]/10000000000) - np.exp((stationary[col]/10000000000).shift(1))
        adf_test(tested_series.dropna())
        kpss_test(tested_series.dropna())
        if (adf_list[-1]==True) & (kpss_list[-1]==True):
          exit=True
      if exit==False:
        for i in [7,30,365]:
          new_col = f'd_{i}_' + col
          tested_series = stationary[col].diff(periods=i) 
          adf_test(tested_series.dropna())
          kpss_test(tested_series.dropna())
          if (adf_list[-1]==True) & (kpss_list[-1]==True):
            exit=True
            break
      if exit==True:
        stationary[new_col] = tested_series
      else:
        persistent_non_stationary.append(col)
  stationary.dropna(inplace=True)
  cols_to_drop = [x for x in stationary.columns if x in non_stationary_cols]
  stationary.drop(cols_to_drop, axis=1, inplace=True)
  stationary_etfs[etf] = stationary

Persistent non-stationary vars are not used in the models.

# Saving Sheets to Gdrive

In [ ]:
for etf in keys:
  path = f'/content/drive/My Drive/Bocci_Machine_Learning_Returns/Data/{etf}.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    stationary_etfs[etf].to_csv(f)

  path = f'/content/drive/My Drive/Bocci_Machine_Learning_Returns/Data/full_describe_{etf}.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    full_dataset_description[etf].to_csv(f)

  path = f'/content/drive/My Drive/Bocci_Machine_Learning_Returns/Data/base_describe_{etf}.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    base_description[etf].to_csv(f)